In [5]:
import math
import numpy as np

In [51]:
class bs_bin_tree:

    def __init__(self,T,s0,r,sigma,c,K,n):
        self.T = T
        self.r = r
        self.c = c
        self.sigma = sigma
        self.K = K
        self.s0 = s0
        self.n = n
        self.u = math.exp(self.sigma*np.sqrt(self.T/self.n))
        self.q = (math.exp((self.r-self.c)*T/self.n)-(1/self.u))/(self.u-(1/self.u))
        self.R = math.exp(self.r*self.T/self.n)
        self.__print_param__()

    def __print_param__(self):
        print('Time',self.T)
        print('Starting Price',self.s0)
        print('r',self.r)
        print('volatility',self.sigma)
        print('dividend yield',self.c)
        print('strike',self.K)
        print('# period',self.n)
        
    
    def generate_price(self):
        arr=[[self.s0]]
        for i in range(self.n):
            arr_to_add=[]
            for j in range(len(arr[i])):
                arr_to_add.append(arr[i][j]/self.u)
                if j == (len(arr[i])-1):
                    arr_to_add.append(arr[i][j]*self.u)
            arr.append(arr_to_add)
        return arr

    def neutral_pricing(self,p1,p2):
        price = ((1-self.q)*p1 + (self.q)*p2)/self.R
        return price

    def eu_call(self):
        arr = self.generate_price()
        arr_rev = arr[::-1]
        res=[]
        for i in range(len(arr_rev)):
            res_to_add = []
            for j in range(len(arr_rev[i])):
                if i == 0:
                    a = max(arr_rev[i][j]-self.K,0)
                    res_to_add.append(a)
                else:
                    price = self.neutral_pricing(res[i-1][j], res[i-1][j+1])
                    #a = max(arr_rev[i][j]-strike,0)
                    #a = max(a,price)
                    a = price
                    res_to_add.append(a)
                
            res.append(res_to_add)
        return res[::-1]

    def us_call(self):
        arr = self.generate_price()
        arr_rev = arr[::-1]
        res=[]
        for i in range(len(arr_rev)):
            res_to_add = []
            for j in range(len(arr_rev[i])):
                if i == 0:
                    a = max(arr_rev[i][j]-self.K,0)
                    res_to_add.append(a)
                else:
                    price = self.neutral_pricing(res[i-1][j], res[i-1][j+1])
                    a1 = max(arr_rev[i][j]-self.K,0)
                    a = max(a1,price)
                    res_to_add.append(a)
                
            res.append(res_to_add)
        return res[::-1]

    def us_call_price(self):
        return self.us_call()[0][0]

    def us_put(self):
        arr = self.generate_price()
        arr_rev = arr[::-1]
        res=[]
        for i in range(len(arr_rev)):
            res_to_add = []
            for j in range(len(arr_rev[i])):
                if i == 0:
                    a = max(self.K-arr_rev[i][j],0)
                    res_to_add.append(a)
                else:
                    price = self.neutral_pricing(res[i-1][j], res[i-1][j+1])
                    a1 = max(self.K - arr_rev[i][j],0)
                    a = max(a1,price)
                    res_to_add.append(a)
                
            res.append(res_to_add)
        return res[::-1]

    def us_put_price(self):
        return self.us_put()[0][0]

    def us_put_early_ex(self):
        early_ex = False
        early_ex_earning = 0
        early_ex_time = self.n
        arr = self.generate_price()
        arr_rev = arr[::-1]
        res=[]
        for i in range(len(arr_rev)):
            res_to_add = []
            for j in range(len(arr_rev[i])):
                if i == 0:
                    a = max(self.K-arr_rev[i][j],0)
                    res_to_add.append(a)
                else:
                    price = self.neutral_pricing(res[i-1][j], res[i-1][j+1])
                    a1 = max(self.K-arr_rev[i][j],0)
                    if a1 > price:
                        if early_ex_time == self.n - i:
                            early_ex_earning = max(early_ex_earning,a1)
                        else:
                            early_ex_earning = a1
                        early_ex =True
                        early_ex_time = self.n - i


                    a = max(a1,price)
                    res_to_add.append(a)
                
            res.append(res_to_add)
        return {early_ex_time:early_ex_earning} if early_ex == True else False



In [52]:
bin = bs_bin_tree(.25,100,0.02,0.3,0.01,110,15)

Time 0.25
Starting Price 100
r 0.02
volatility 0.3
dividend yield 0.01
strike 110
# period 15


In [53]:
# answer to first question
bin.us_call_price()

2.6040771329665713

In [54]:
bin.us_put()

[[12.359784797284878],
 [15.07698187140818, 9.567862168218058],
 [18.036477300042694, 12.037190792038981, 7.029497833457652],
 [21.165110262676198, 14.82437990265578, 9.172917359198392, 4.825290601977193],
 [24.380542405888693,
  17.8656773365925,
  11.700114280624874,
  6.574647728003668,
  3.025704141468864],
 [27.605307891822548,
  21.073667480875617,
  14.571680361464452,
  8.748655409935077,
  4.338609023082057,
  1.6746985095614002],
 [30.735434694373126,
  24.35157360133246,
  17.709788198153895,
  11.347472771456978,
  6.076287947791879,
  2.5507284169335844,
  0.7730130403816153],
 [33.74664978611615,
  27.605307891822548,
  21.00766988251785,
  14.323045236876627,
  8.288588237426527,
  3.8004482583419854,
  1.2645184991695022,
  0.2670004892894602],
 [36.64347055432029,
  30.735434694373126,
  24.35157360133246,
  17.575730094233784,
  10.980588503363494,
  5.5198768534822635,
  2.0310117626694213,
  0.47544166027796597,
  0.05236586972748448],
 [39.430242773318895,
  33.746

In [55]:
bin.us_put_early_ex()

{5: 27.605307891822548}